In [92]:
from typing import List, Optional
from pydantic import BaseModel, validator
from functools import total_ordering


@total_ordering
class BoundingBox(BaseModel):
    """Bounding Box value type holds sizes in all three dimensions (in mm).
    These sizes indicate the maximum size that an entity takes in each of the three dimensions.
    """

    length: Optional[int] = 0
    width: Optional[int] = 0
    height: Optional[int] = 0

    @validator("length", "width", "height")
    def sizes_must_be_non_negative(cls, value: int) -> int:
        """ensure that none of the sizes are negative numbers"""
        if value < 0:
            raise ValueError("each of the sizes must be positive")
        return value

    @property
    def base_area(self) -> int:
        """calculate the base area of the bouding box, assuming that the dimensions length and width lie at the base of the box. If the box is rotated in an axis such that the base area is not represented by the rectangle formed by the length and the width. This function would return incorrect base_area"""
        return self.length * self.width
    
    @property
    def volume(self) -> int:
        """calculate volume of the cuboidal bounding box"""
        return self.length * self.width * self.height
    
    def __lt__(self, other: 'BoundingBox') -> bool:
        """a bounding box is less than another bounding box Only IF, one can contain another inside it. It means all the sizes of a bounding box must be less than that of another for it to be lesser"""
        return (self.length < other.length) and (self.width < other.width) and (self.height < other.height)


class Offset(BaseModel):
    """Offset from a given origin (0,0,0) in mm."""

    x: int = 0
    y: int = 0
    z: int = 0


class Constraints(BaseModel):
    """Hold Bounding Boxes for Available Working Region, along with offsets that may further shrink this volume.
    Available Working Region is the region available for placement of modules.
    """

    working_region_bounding_box: BoundingBox
    pipette_bounding_box: BoundingBox
    pipette_offset: Offset


class Module(BaseModel):
    """Any module must provide a name and a bounding box."""

    name: str
    bounding_box: BoundingBox
    offset: Optional[Offset]


class Layout(BaseModel):
    """A layout model is the top most model for parsing and representing the contents of gero's layout file
    This contains both the constraints that limit layoutability, as well as bounding box and positional concerns of the modules that are to be provisioned in the layout.
    """

    constraints: Constraints
    modules: List[Module]

    def is_feasible(self) -> bool:
        return self.bounding_box() < self.constraints.working_region_bounding_box

    @property
    def length(self) -> int:
        return sum([m.bounding_box.length for m in self.modules])

    @property
    def width(self) -> int:
        return max([m.bounding_box.height for m in self.modules])
    
    @property
    def height(self) -> int:
        return max([m.bounding_box.height for m in self.modules])

    def bounding_box(self) -> BoundingBox:
        return BoundingBox(length=self.length, width=self.width, height=self.height)

In [94]:
from json import load
from pydantic import ValidationError
# from devtools import debug

file = "../sample_config_files/.gero/computed_layout.json"

with open(file=file) as f:
    data = load(f)
    # debug(data)

    try:
        layout = Layout(**data)
    except ValidationError as e:
        print(e)

    debug(layout.is_feasible())
    
    # debug(layout.constraints)
    # debug(layout.modules)


/var/folders/fg/1szv954j4cq_g_w5qvrpm1340000gn/T/ipykernel_95334/2877144228.py:16 <cell line: 7>
    layout.is_feasible(): True (bool)


In [89]:
wbox = layout.constraints.pipette_bounding_box
print(type(wbox))

print(f"area={wbox.base_area}mm^2, vol={wbox.volume}mm^3")

<class '__main__.BoundingBox'>
area=165750mm^2, vol=31492500mm^3


In [91]:
constraint = Constraints(
    working_region_bounding_box=BoundingBox(), 
    pipette_bounding_box=BoundingBox(), 
    pipette_offset=Offset()
    )

modules = []

layout = Layout(constraints=constraint, modules=modules)